In [9]:
import os
import glob
import csv
from PIL import Image
from pillow_heif import register_heif_opener
register_heif_opener()

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models


In [10]:
class MyDigitDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.image_paths = []
        for ext in ('*.jpg', '*.jpeg', '*.png', '*.heic', '*.jfif'):
            self.image_paths.extend(glob.glob(os.path.join(root_dir, ext)))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            label = int(os.path.basename(img_path).split('_')[0])
            if self.transform:
                image = self.transform(image)
            return image, label
        except:
            print(f"Lỗi ảnh: {img_path}")
            return None

class MyTestDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.transform = transform
        self.image_paths = []
        for ext in ('*.jpg', '*.jpeg', '*.png', '*.heic', '*.jfif'):
            self.image_paths.extend(glob.glob(os.path.join(folder, '**', ext), recursive=True))

        valid_paths = []
        for path in self.image_paths:
            try:
                Image.open(path).verify()
                valid_paths.append(path)
            except:
                print(f"Ảnh lỗi (bỏ qua): {path}")
        self.image_paths = valid_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_path


In [11]:
transformtrain = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transformtest = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [12]:
root_path = r'D:\Train2'
test_folder = r'D:\Test\data.2025'
traindata = MyDigitDataset(root_path, transform=transformtrain)
traindata = [item for item in traindata if item is not None]
train_loader = DataLoader(traindata, batch_size=64, shuffle=True, num_workers=0)

testdata = MyTestDataset(test_folder, transform=transformtest)
test_loader = DataLoader(testdata, batch_size=64, shuffle=False, num_workers=0)

print("Train samples:", len(traindata))
print("Test samples:", len(testdata))


Ảnh lỗi (bỏ qua): D:\Test\data.2025\2c18ee0e7cea8354149df435532d74ae.jpeg
Ảnh lỗi (bỏ qua): D:\Test\data.2025\3a816aa78f56749a0822d700ff560924.jpeg
Ảnh lỗi (bỏ qua): D:\Test\data.2025\4abc1b5dcf1be1de6503dc072e132fa0.jpeg
Ảnh lỗi (bỏ qua): D:\Test\data.2025\70ebf0bec317006017a54d6c9172af45.jpeg
Ảnh lỗi (bỏ qua): D:\Test\data.2025\8eeee227b3f244e980b747387bc79bf2.jpeg
Ảnh lỗi (bỏ qua): D:\Test\data.2025\9e748b9617e26b90011f8d7c3f8a7eee.jpeg
Ảnh lỗi (bỏ qua): D:\Test\data.2025\a008f795ae8498751f2e5feeb0b73387.jpeg
Ảnh lỗi (bỏ qua): D:\Test\data.2025\ce2388179ec73203a60d8efedd04e0f1.jpeg
Ảnh lỗi (bỏ qua): D:\Test\data.2025\d4b080c6ce4933f1199fad7f75e7d112.jpeg
Ảnh lỗi (bỏ qua): D:\Test\data.2025\e21dcb1cb0571a21a013b4232d1b6a13.jpeg
Ảnh lỗi (bỏ qua): D:\Test\data.2025\f7679411e290bfd8dc90b3eadb57460a.jpeg
Ảnh lỗi (bỏ qua): D:\Test\data.2025\ffd4f9df7163f55474a0182265c8d810.jpeg
Train samples: 5473
Test samples: 9975


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [14]:
def evaluate(model, dataloader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total if total > 0 else 0


In [15]:
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    train_acc = evaluate(model, train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {train_acc*100:.2f}%")


Epoch 1/50, Loss: 0.8215, Accuracy: 83.25%
Epoch 2/50, Loss: 0.3637, Accuracy: 90.08%
Epoch 3/50, Loss: 0.2354, Accuracy: 90.88%
Epoch 4/50, Loss: 0.1966, Accuracy: 93.81%
Epoch 5/50, Loss: 0.1771, Accuracy: 90.15%
Epoch 6/50, Loss: 0.1730, Accuracy: 95.10%
Epoch 7/50, Loss: 0.1281, Accuracy: 97.26%
Epoch 8/50, Loss: 0.0817, Accuracy: 97.37%
Epoch 9/50, Loss: 0.0793, Accuracy: 97.17%
Epoch 10/50, Loss: 0.1108, Accuracy: 96.62%
Epoch 11/50, Loss: 0.1126, Accuracy: 97.99%
Epoch 12/50, Loss: 0.0709, Accuracy: 97.83%
Epoch 13/50, Loss: 0.0769, Accuracy: 97.75%
Epoch 14/50, Loss: 0.0540, Accuracy: 97.02%
Epoch 15/50, Loss: 0.0646, Accuracy: 94.92%
Epoch 16/50, Loss: 0.0829, Accuracy: 97.81%
Epoch 17/50, Loss: 0.0766, Accuracy: 97.30%
Epoch 18/50, Loss: 0.0923, Accuracy: 94.28%
Epoch 19/50, Loss: 0.0718, Accuracy: 98.43%
Epoch 20/50, Loss: 0.0501, Accuracy: 98.08%
Epoch 21/50, Loss: 0.0441, Accuracy: 98.63%
Epoch 22/50, Loss: 0.0470, Accuracy: 97.94%
Epoch 23/50, Loss: 0.0773, Accuracy: 94.2

In [16]:
model.eval()
results = []
with torch.no_grad():
    for images, img_paths in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        preds = preds.cpu().numpy()
        for path, pred in zip(img_paths, preds):
            filename = os.path.basename(path)
            results.append([filename, int(pred)])

with open('WECODE_RESNET18(4).csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['filename', 'prediction'])
    writer.writerows(results)
